# Scraping rightmove.co.uk data

Data is scraped from [Rightmove](https://www.rightmove.co.uk/property-to-rent.html). Data is limited to properties available for rent in London.

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from time import sleep
from tqdm import tqdm

In [2]:
base_url = 'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E87490'
page_code = '&index='
url_end = '&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='

listing = []
first_page = requests.get(f"{base_url}{url_end}")
soup = BeautifulSoup(first_page.text, "html.parser")
listing.append(soup.find_all(class_="propertyCard-wrapper"))

In [3]:
for i in tqdm(range(24,984,24)):
    following_pages = requests.get(f"{base_url}{page_code}{i}{url_end}")
    soup = BeautifulSoup(following_pages.text, "html.parser")
    listing.append(soup.find_all(class_="propertyCard-wrapper"))
    sleep(5)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [07:36<00:00, 11.50s/it]


In [4]:
%%capture

with open("london-rent.csv", "w", encoding='utf-8') as file:
    writer = csv.writer(file);
    writer.writerow(["title", "address", "price", "phone", "branch"])

    for ad in listing:
        for i in ad:

            title = str(i.find(class_="propertyCard-title").get_text()).strip()

            address = str(i.find(class_="propertyCard-address").get_text()).strip()

            price = str(i.find(class_="propertyCard-priceValue").get_text().strip())

            phone = str(i.find(class_="propertyCard-contactsPhoneNumber").get_text()).strip()

            branch = str(i.find(class_="propertyCard-branchSummary-branchName").get_text()).strip()

            writer.writerow([title,address,price,phone,branch])

file.close()

In [5]:
import pandas as pd

pd.read_csv("london-rent.csv").head(10)

,title,address,price,phone,branch
0,2 bedroom apartment,"Onega Gate, SE16","£1,712 pcm",020 8012 2917,"by Hamptons International Lettings, Tower Bridge"
1,4 bedroom semi-detached house,"Perth Road, London, N22","£2,400 pcm",020 8012 3106,"by PropertyNet, Crouch End"
2,3 bedroom ground floor flat,"Rosebery Gardens, London, N8","£1,863 pcm",020 8012 3106,"by PropertyNet, Crouch End"
3,3 bedroom flat,"Tottenham Lane,London,N8","£1,500 pcm",020 8012 3106,"by PropertyNet, Crouch End"
4,2 bedroom apartment,"Distillery Wharf, Fulham Reach","£3,683 pcm",020 8012 5410,"by Martin & Co, London Riverside - Lettings & ..."
5,1 bedroom apartment,"Brierly Gardens, Bethnal Green, E2","£1,400 pcm",020 8012 1071,"by Peach Properties, Shoreditch"
6,3 bedroom flat,"Juliet House, Hoxton, N1","£1,900 pcm",020 8012 5452,"by Hawkes Property Group, London"
7,2 bedroom flat,"Clifton Gardens, Golders Green","£1,603 pcm",020 8012 2003,"by Kingsleys Estates, Golders Green - Lettings"
8,2 bedroom flat,"Chilton Grove, London, SE8","£1,350 pcm",020 8012 7269,"by Hamilton Leigh Estates UK, Barking and Dage..."
9,1 bedroom flat,"Fonthill Road, London, N4","£1,400 pcm",0115 798 0536,"by Ion Property Management, London"
